##  필요한 패키지 설치

In [ ]:
%pip install transformers datasets accelerate trl peft hf_transfer langchain python-dotenv

In [ ]:
import torch

torch.__version__

# Dataset Loading

In [ ]:
import os
from huggingface_hub import login
from dotenv import load_dotenv

print(load_dotenv("env"))

login(os.getenv('HUGGINGFACE_API_KEY'))

In [ ]:
from datasets import load_dataset

user_id = "kgmyh"  # 본인 Huggingface 사용자명 입력
data_id = f"{user_id}/naver_economy_news_stock_instruct_dataset-100_samples"
dataset = load_dataset(data_id)

dataset

In [ ]:
train_set = dataset['train']
test_set = dataset['test']

# sLLM Model Load

## Base Model Load
- 한국어를 충분히 학습한 모델을 선택한다.

- kakao의 kanana 모델을 base 모델로 파인튜닝을 진행한다.
  - https://tech.kakao.com/author/Kanana
  - https://github.com/kakao/kanana

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "kakaocorp/kanana-nano-2.1b-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype=torch.bfloat16,
    attn_implementation="eager" 
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

### Base모델 사용

In [ ]:
############################################################################################################################################### 
#  입력 프롬프트 생성
#
#  - kanana, Llama 동일한 프롬프트 형식
#    - Instruction 모델이 학습할 때 사용한 prompt 형식에 맞춰 입력데이터를 변환을 해야 한다.
#  - `tokenizer.apply_chat_template()`: 
#       - {"role":"역할", "content":"content"} 구조로 입력을 하면 실제 모델의 모델의 입력형식으로 변환해 주는 메소드.
###############################################################################################################################################

content = "오늘 서울 날씨 어때요?"
message = [  
        {"role": "system", "content": "당신은 인공지능 날씨 예보관입니다."},
        {"role": "user", "content": content},
]

# 모델 입력 형식에 맞게 프롬프트 변환.
prompt = tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=True
)
print(prompt)



In [ ]:
########################
# 토큰화
########################

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
print(inputs)

In [ ]:
inputs.keys()

In [ ]:
########################################## 
# 답변 생성
## 생성모델: model.generate() 메소드 사용
##########################################
import torch

model.eval()
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=100, # 답변 토큰 수 제한
        do_sample=True,     # True: 확률 분포에 따라 다음 토큰을 무작위로 선택(확률 기준: top_p, temperature), False: 가장 높은 확률의 토큰만 선택
        top_p=0.95,         # 다음에 올 확률 순으로 토큰들 정렬 → 누적 → 0.95(지정한값) 도달 시점까지 상위토큰만 남긴다. 낮으면(예: 0.5) 창의성↓, 너무 높으면(1.0) 창의성↑
        temperature=0.8,    # 토큰의 다양성을 지정. 낮을수록(0에 가까울수록) 높은 확률의 토큰을 더욱 선택. 낮으면(예: 0.5) 너무 높으면(1.0) 창의성↑
    )

In [ ]:
output

In [ ]:
print(tokenizer.decode(output[0]))

In [ ]:
inputs["input_ids"].shape

In [ ]:
# 답변 부분만 잘라서 디코딩
response = tokenizer.decode(output[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
print(response)

# 학습 전에 답변 결과 확인

### 추론용 프롬프트 만들기

In [ ]:
######################################
#  System 프롬프트
######################################

system_prompt = '''# Instruction
당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.
사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.

## 분석 기준
1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.
2. 영향을 준다면 다음 항목을 출력하세요.
   - `"is_stock_related": true`
   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**
   - 뉴스에 **부정적** 영향을 받는 **회사이름들**
   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**
     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.
     - `None`, 유추, 추정, 일반 논평 금지합니다.
   - 뉴스 요약 (3줄 이내)
3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.
   - `"is_stock_related": false`
   - 뉴스 요약 (3줄 이내)

# 출력 지시사항 (Output Indicator)
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"is_stock_related": {"description": "한국 주식과 관련있는 뉴스인지 여부", "title": "Is Stock Related", "type": "boolean"}, "positive_stocks": {"description": "뉴스기사에 긍정적인 영향을 받는 회사들의 이름들.", "items": {"type": "string"}, "title": "Positive Stocks", "type": "array"}, "positive_reason": {"description": "뉴스내용 중 positive_stocks에 있는 각 회사들에 긍정적 영향을 주는 내용. {\"회사이름\":\"긍정적인 이유\"}", "items": {"additionalProperties": {"type": "string"}, "type": "object"}, "title": "Positive Reason", "type": "array"}, "negative_stocks": {"description": "뉴스기사에 부정적인 영향을 받는 회사들의 이름들.", "items": {"type": "string"}, "title": "Negative Stocks", "type": "array"}, "negative_reason": {"description": "뉴스내용 중 negative_stocks에 있는 각 회사들에 부정적 영향을 주는 내용. {\"회사이름\":\"부정적인 이유\"}", "items": {"additionalProperties": {"type": "string"}, "type": "object"}, "title": "Negative Reason", "type": "array"}, "summary": {"description": "뉴스기사 요약", "title": "Summary", "type": "string"}}, "required": ["is_stock_related", "positive_stocks", "positive_reason", "negative_stocks", "negative_reason", "summary"]}
```

## 출력 조건:
- 뉴스에 영향을 받은 회사들은 **반드시 한국 증시에 상장된 종목** 이어야 합니다.
- 뉴스에 있는 내용만 출력결과에 포함시킵니다.
- 긍정/부정 종목은 실제 뉴스기사에 영향을 받는 회사들만 포함하세요.
- 모든 문자열은 큰따옴표(`"`)로 감쌉니다.
- 문자열 안에 따옴표가 필요하면 작은따옴표(`'`)를 사용합니다.
- 모든 키(Key)는 출력 지시사항에 명시된 property들과 정확히 일치해야 합니다.
- `"positive_reasons"` 및 `"negative_reasons"`의 값은 `None`이 될 수 없습니다.
- json format을 잘 지켜 응답데이터를 만듭니다. 배열이나 object의 마지막 항목 뒤에 `,` 를 붙이지 마세요.
- 오직 유효한 JSON 문자열(UTF-8, RFC8259 준수)만 출력합니다.
- 절대 다른 텍스트, 주석, 설명, 코드 블록 표기(```), 또는 따옴표 외의 문자열을 추가하면 안 됩니다.

## 출력 예시 (Examples)

### 뉴스가 특정 주식종목들에 **긍정적 영향이 주는 경우**:
{'is_stock_related': True,
 'negative_reasons': [],
 'negative_stocks': [],
 'positive_reasons': [{'세라젬': '루게릭병 환우 지원 캠페인 후원과 의료가전 지원 등 사회공헌활동을 통해 기업 이미지와 브랜드 가치가 긍정적으로 부각됨'}],
 'positive_stocks': ['세라젬'],
 'summary': '세라젬이 루게릭병 환우를 위한 아이스버킷 챌린지 런 행사를 후원하며 의료가전과 건강기능식품 등을 지원했다. 캠페인은 루게릭병 환우 지원과 기부 문화 확산을 목표로 한다. 세라젬은 다양한 사회공헌활동을 지속하고 있다.'
}

### 뉴스의 내용이 특정 주식종목들에 **부정적 영향이 주는 경우**:
{
    "is_stock_related": true,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [
        "포스코",
        "현대제철"
    ],
    "negative_reasons": [
        {"포스코": "정부가 수입규제국 조사에 적극 대응하고 비관세장벽 해소를 위해 민관 협력 강화 방침을 밝혀 철강 분야에서 수출 피해 최소화 기대"},
        {"현대제철": "철강·금속 품목에 대한 수입규제 대응 강화로 불합리한 무역제한 조치 개선 가능성이 높아져 수출 환경 개선 기대"}
    ],
    "summary": "산업부는 수입 규제국의 조사에 대응하고 비관세장벽 해소를 위한 협의를 진행했다. 규제 대상 국가는 26개국, 건수는 199건에 달한다."
}

### **뉴스기사가 주식 종목과 관련 없는 경우**:
{
    "is_stock_related": false,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [],
    "negative_reasons": [],
    "summary": "정황근 농림축산식품부 장관이 단순가공식품 부가가치세 면제 시행 상황을 점검했다. 된장, 고추장 코너를 방문하며 현장을 살폈다."
}'''


In [ ]:
################################################################
# 추론할 1개의 샘플 데이터 생성
# 뉴스 기사 제목 + "\n\n" + 뉴스 기사 내용
################################################################

idx = 5

sample = dataset['train'][idx]
user_input = sample["title"]+"\n\n"+sample['document']
prompt = [
    {"role":"system", "content":system_prompt},
    {"role":"user", "content": user_input}
]
prompt

In [ ]:
#########################################################
# Pipeline을 이용해 실행
#  - task: text-generation
#  - pipeline은 (role base) chat format 으로 입력한다. 
#########################################################
from transformers import pipeline

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer )
res = pipe(prompt,  max_new_tokens=200)

In [ ]:
print(res)

In [ ]:
response = res[0]['generated_text'][-1]
print(response['content'])

# 파인 튜닝 

## 데이터셋 만들기

### 프롬프트 생성
- LLM 모델은 학습할때 사용한 프롬프트 형식이 있다.
    - 모델 마다 형식이 다르기 때문에 확인이 필요하다.
    - 모델이 사용한 tokenizer의 chat_template 속성을 이용해 조회할 수 있다.
      - `tokenizer.chat_template`
      - `jija2` 템플릿 엔진 문법으로 작성됨.
- 학습데이터를 LLM의 프롬프트 형식으로 생성한다.

### Llama 모델의 chat template 형식
```
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
[시스템 역할 지침]<|eot_id|>
<|start_header_id|>user<|end_header_id|>
[유저 질문]<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
[모델의 답변]<|eot_id|>
```
- `<|begin_of_text|>`: 시퀀스의 시작을 나타내는 토큰.
- `<|start_header_id|>ROLE<|end_header_id|>`: Role(발화자) 지정 - system | user | assistant
- `메세지 내용<|eot_id|>`:  Role 메세지. 시퀀스 종료를 나타내는 토큰(`<|eot_id|>`)

#### 예
```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 인공지는 날씨 예보관입니다.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
오늘 서울 날씨 어때요?<|eot_id|>
```
> ### Gemma format chat template 형식
> 
> ```xml
> <bos><start_of_turn>Role
> {사용자 입력-input}<end_of_turn>
> <start_of_turn>model
> {AI 답변-label}<end_of_turn>
> ```
> -	`<bos>`: 시퀀스의 시작을 나타내는 토큰.
> -	`<start_of_turn>`: 각 대화의 시작을 나타낸다.
> -	`Role`: Role(발화자) 지정 - **user** | **model** 
>   - gemma는 user와 model 두가지 role을 이용해 instruction 모델을 학습함.
>   - gemma 는 system role을 사용하지 않는다. 그래서 **system prompt는 user role의 content에 넣어준다.**
> - `메세지`
>   - Role 다음 줄에 이어서 메세지를 입력한다.
> - `<end_of_turn>`: 대화의 끝을 나타냅니다.
> - 예
> ```xml
> <bos><start_of_turn>user
> AI에 대해 설명해주세요.<end_of_turn>
> <start_of_turn>model
> AI은 컴퓨터 시스템이 인간의 지능적 기능을 모방하여 데이터를 처리하고 의사결정을 수행하는 기술입니다.<end_of_turn>
> ```

> ### Alapaca format chat template
> ```
> ### Instruction:
> [시스템 역할 지침]
> 
> [유저 질문]
> 
> ### Response:
> [모델의 답변]
> ```
> - `### Instruction`: - 사용자 입력/질문
> - `### Response`: - 모델의 답변
> - 특수 토큰 대신 마크다운 스타일의 헤더 사용
> - 보통 맨 앞에 표준 instruction 문구 포함


### InputPromptCreator(프롬프트 생성 클래스) 정의

- `create_pipeline_prompt()`:
    - 뉴스기사 제목, 뉴스기사내용을 받아서 파이프라인에 입력할 chat 형식 프롬프트를 생성한다.
- `create_generate_prompt:()`:
    - 뉴스기사 제목, 뉴스기사내용을 받아서 모델의 자체 chat 형식의 프롬프트를 생성한다.
    - **system_prompt, user_input** 으로 구성된 prompt 생성
- `create_train_prompt()`:
    - Dataset의 개별 데이터를 입력받아서 모델 학습을 위한 chat 프롬프트 생성.
    - **system_prompt, user_input, label** 로 구성된 prompt 생성

In [ ]:
####################################################################
# 학습 용 프롬프트 생성 함수
####################################################################


from textwrap import dedent
class InputPromptCreator:
    """모델별 형식에 맞춰 입력 프롬프트를 생성한다.
    """
    def __init__(self, tokenizer=None):
        """
        모델의 chat template을 제공하는 tokenizer와 system 프롬프트를 받아서 초기화
        Args:
            tokenizer : 모델의 chat template을 제공하는 tokenizer
        """
        self.tokenizer = tokenizer
        self.system_prompt = dedent('''
        # Instruction
        당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.
        사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.
        
        ## 분석 기준
        1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.
        2. 영향을 준다면 다음 항목을 출력하세요.
           - `"is_stock_related": true`
           - 뉴스에 **긍정적** 영향을 받는 **회사이름들**
           - 뉴스에 **부정적** 영향을 받는 **회사이름들**
           - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**
             - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.
             - `None`, 유추, 추정, 일반 논평 금지합니다.
           - 뉴스 요약 (3줄 이내)
        3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.
           - `"is_stock_related": false`
           - 뉴스 요약 (3줄 이내)
        
        # 출력 지시사항 (Output Indicator)
        The output should be formatted as a JSON instance that conforms to the JSON schema below.
        
        As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
        the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.
        
        Here is the output schema:
        ```
        {"properties": {"is_stock_related": {"description": "한국 주식과 관련있는 뉴스인지 여부", "title": "Is Stock Related", "type": "boolean"}, "positive_stocks": {"description": "뉴스기사에 긍정적인 영향을 받는 회사들의 이름들.", "items": {"type": "string"}, "title": "Positive Stocks", "type": "array"}, "positive_reason": {"description": "뉴스내용 중 positive_stocks에 있는 각 회사들에 긍정적 영향을 주는 내용. {\"회사이름\":\"긍정적인 이유\"}", "items": {"additionalProperties": {"type": "string"}, "type": "object"}, "title": "Positive Reason", "type": "array"}, "negative_stocks": {"description": "뉴스기사에 부정적인 영향을 받는 회사들의 이름들.", "items": {"type": "string"}, "title": "Negative Stocks", "type": "array"}, "negative_reason": {"description": "뉴스내용 중 negative_stocks에 있는 각 회사들에 부정적 영향을 주는 내용. {\"회사이름\":\"부정적인 이유\"}", "items": {"additionalProperties": {"type": "string"}, "type": "object"}, "title": "Negative Reason", "type": "array"}, "summary": {"description": "뉴스기사 요약", "title": "Summary", "type": "string"}}, "required": ["is_stock_related", "positive_stocks", "positive_reason", "negative_stocks", "negative_reason", "summary"]}
        ```
        
        ## 출력 조건:
        - 뉴스에 영향을 받은 회사들은 **반드시 한국 증시에 상장된 종목** 이어야 합니다.
        - 뉴스에 있는 내용만 출력결과에 포함시킵니다.
        - 긍정/부정 종목은 실제 뉴스기사에 영향을 받는 회사들만 포함하세요.
        - 모든 문자열은 큰따옴표(`"`)로 감쌉니다.
        - 문자열 안에 따옴표가 필요하면 작은따옴표(`'`)를 사용합니다.
        - 모든 키(Key)는 출력 지시사항에 명시된 property들과 정확히 일치해야 합니다.
        - `"positive_reasons"` 및 `"negative_reasons"`의 값은 `None`이 될 수 없습니다.
        - json format을 잘 지켜 응답데이터를 만듭니다. 배열이나 object의 마지막 항목 뒤에 `,` 를 붙이지 마세요.
        - 오직 유효한 JSON 문자열(UTF-8, RFC8259 준수)만 출력합니다.
        - 절대 다른 텍스트, 주석, 설명, 코드 블록 표기(```), 또는 따옴표 외의 문자열을 추가하면 안 됩니다.
        
        ## 출력 예시 (Examples)
        
        ### 뉴스가 특정 주식종목들에 **긍정적 영향이 주는 경우**:
        {'is_stock_related': True,
         'negative_reasons': [],
         'negative_stocks': [],
         'positive_reasons': [{'세라젬': '루게릭병 환우 지원 캠페인 후원과 의료가전 지원 등 사회공헌활동을 통해 기업 이미지와 브랜드 가치가 긍정적으로 부각됨'}],
         'positive_stocks': ['세라젬'],
         'summary': '세라젬이 루게릭병 환우를 위한 아이스버킷 챌린지 런 행사를 후원하며 의료가전과 건강기능식품 등을 지원했다. 캠페인은 루게릭병 환우 지원과 기부 문화 확산을 목표로 한다. 세라젬은 다양한 사회공헌활동을 지속하고 있다.'
        }
        
        ### 뉴스의 내용이 특정 주식종목들에 **부정적 영향이 주는 경우**:
        {
            "is_stock_related": true,
            "positive_stocks": [],
            "positive_reasons": [],
            "negative_stocks": [
                "포스코",
                "현대제철"
            ],
            "negative_reasons": [
                {"포스코": "정부가 수입규제국 조사에 적극 대응하고 비관세장벽 해소를 위해 민관 협력 강화 방침을 밝혀 철강 분야에서 수출 피해 최소화 기대"},
                {"현대제철": "철강·금속 품목에 대한 수입규제 대응 강화로 불합리한 무역제한 조치 개선 가능성이 높아져 수출 환경 개선 기대"}
            ],
            "summary": "산업부는 수입 규제국의 조사에 대응하고 비관세장벽 해소를 위한 협의를 진행했다. 규제 대상 국가는 26개국, 건수는 199건에 달한다."
        }
        
        ### **뉴스기사가 주식 종목과 관련 없는 경우**:
        {
            "is_stock_related": false,
            "positive_stocks": [],
            "positive_reasons": [],
            "negative_stocks": [],
            "negative_reasons": [],
            "summary": "정황근 농림축산식품부 장관이 단순가공식품 부가가치세 면제 시행 상황을 점검했다. 된장, 고추장 코너를 방문하며 현장을 살폈다."
        }''')

    def create_pipeline_prompt(self, news_title:str, news_document:str) -> list[dict]:
        """파이프라인에 입력할 chat 형식 프롬프트생성한다.
        ```
        [
            {"role":"system", "content":시스템프롬프트},
            {"role":"user", "content": news_title+"\n\n"+news_document}
        ]
        ```
        """
        news = news_title+"\n\n"+news_document
        message = [
            {"role":"system", "content":self.system_prompt},
            {"role":"user", "content": news}
        ]
        return message

    def create_generate_prompt(self, news_title:str, news_document:str) -> str:
        """뉴스 제목과 내용을 받아서 tokenizer를 이용해 모델의 자체 입력 chat 형식의 프롬프트를 생성한다.
        tokenizer.apply_chat_template() 메소드의 결과를 반환
        ```
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {시스템 프롬프트}<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        {입력 - title+document}<|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        ```
        """
        if self.tokenizer is None:
            raise Exception("Tokenizer가 없습니다. generate_prompt를 사용하려면 모델의 tokenizer가 필요합니다.")
        message = self.create_pipeline_prompt(news_title, news_document)
        prompt =  tokenizer.apply_chat_template(
            message,
            tokenize=False,
            add_generation_prompt=True
        )
        return prompt

    def create_train_prompt(self, datapoint:dict) -> dict:
        """
        Dataset의 개별 데이터를 입력받아서 모델 학습을 위한 chat 프롬프트생성
        프롬프트 형식
        ```
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {시스템 프롬프트}<|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        {입력 - title+document}<|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        {답변 - Label}<|eot_id|>                       ← create_generate_prompt() 에서 답변이 추가됨
        ```
        Args:
            datapoint (dict): 변환할 데이터
    
        Returns:
            str: 모델 학습을 위한 프롬프트. 
        """
        chat_template = dedent('''
            <|begin_of_text|>
            <|start_header_id|>system<|end_header_id|>
            {system_prompt}<|eot_id|>
            <|start_header_id|>user<|end_header_id|>
            {input_content}<|eot_id|>
            <|start_header_id|>assistant<|end_header_id|>
            {label}<|eot_id|>''')
    
        content = datapoint['title']+"\n"+datapoint['document']
        prompt = chat_template.format(system_prompt=self.system_prompt, 
                                      input_content=content, 
                                      label=datapoint['label'])

        return {"train_prompt":prompt}

In [ ]:
prompt_creator = InputPromptCreator(tokenizer=tokenizer)
prompt_creator.create_pipeline_prompt(">>>>>>>>>뉴스기사제목","<<<<<<<뉴스내용" )

In [ ]:
prompt_creator.create_generate_prompt(">>>>>>>>>뉴스기사제목","<<<<<<<뉴스내용")

In [ ]:
prompt_creator.create_train_prompt(train_set[0])

### InputPromptCreator.create_train_prompt()를 이용해 입력 프롬프트 만들기

In [ ]:
##########################
# Trainset
##########################

llama_input_creator = InputPromptCreator(tokenizer)
# Dataset.map(함수) -> 개별 데이터를 함수에 전달. 
#   함수의 반환값({feature이름:값})을 Dataset에 추가. 
trainset = train_set.map(llama_input_creator.create_train_prompt, remove_columns=list(train_set.features))
trainset

In [ ]:
### # Testset
##########################

testset = test_set.map(llama_input_creator.create_train_prompt, remove_columns=list(test_set.features))
testset

## 파인튜닝

### Data Collator

- 학습 도중 입력 데이터를 받아 전처리하는 함수
  - Dataset -> Data Collator함수 -> 모델
    - 데이터셋에서 모델에 전달되는 batch를 받아서 모델에 입력전에 해야하는 처리를 담당하는 함수(Callable).

- 구현할 내용
  - 모델 chat 형식의 문자열을 transformers 모델에 입력하기 위한 inputs를 만든다.
    
  ```json
    {
      "input_ids":입력 sequence의 토큰 ID들,
      "attention_mask":입력토큰과 padding구분,
      "labels": input_ids에서 답변부분 masking. 답변은 토큰ID 나머지는 -100으로 채운다. 
    }
  ```

In [ ]:
import torch

max_seq_length = 8192

def collate_fn(batch: list[dict]) -> dict[str, torch.Tensor]:
    """모델 학습시 batch를 입력받아 모델 입력에 맞게 처리해서 반환
    Args:
        batch (list[dict]): 배치 데이터, dict: {"train_prompt":input text}

    Returns:
        dict[str, torch.Tensor]: 모델 입력에 맞게 처리된 배치 데이터 (inputs, attention_mask, labels)
    """
    
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for prompt in batch:
        
        text = prompt['train_prompt'].strip()
        
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length, # max_length이하는 자른다. truncation=True
            padding=False,             # padding은 뒤에서 수동으로 처리할 것이기 때문에 padding처리하지 않는다.
            return_tensors=None,       # list로 반환.
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        labels = [-100] * len(input_ids) # 답변 token값을 넣을 리스트. system/user prompt 부분은 -100으로, 답변 부분은 토큰값들로 변경.

        ########################################################
        # chat prompt에서 답변 부분을 찾아서 labels를 구성한다. 
        ########################################################
        assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
        assistant_tokens = tokenizer.encode(assistant_header, add_special_tokens=False)

        eot_token = "<|eot_id|>"
        eot_tokens = tokenizer.encode(eot_token, add_special_tokens=False)

        i = 0
        while i <= len(input_ids) - len(assistant_tokens):
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
                start = i + len(assistant_tokens)
                end = start
                while end <= len(input_ids) - len(eot_tokens):
                    if input_ids[end:end + len(eot_tokens)] == eot_tokens:
                        break
                    end += 1
                for j in range(start, end):
                    labels[j] = input_ids[j]
                for j in range(end, end + len(eot_tokens)):
                    labels[j] = input_ids[j]
                break
            i += 1
        
        ##################################################################
        # 생성된 input_ids, attention_mask, labels를 new_batch에 추가한다.
        ##################################################################
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)
    

    ######################################################
    #  패딩 처리 - 동적 패딩.
    #  -  배치내 입력중 가장 긴 sample에 길이를 맞춘다.
    ######################################################
    max_length = max(len(ids) for ids in new_batch["input_ids"])            
    for i in range(len(new_batch["input_ids"])):
        pad_len = max_length - len(new_batch["input_ids"][i]) 
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * pad_len)
        new_batch["attention_mask"][i].extend([0] * pad_len)
        new_batch["labels"][i].extend([-100] * pad_len)

    # list -> torch.Tensor로 변환.
    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])

    return new_batch

In [ ]:
######################
# 확인
######################
example = [trainset[11], trainset[2], trainset[3]]
batch = collate_fn(example)

print("batch:")
print("input_ids 크기:", batch["input_ids"].shape)
print("attention_mask 크기:", batch["attention_mask"].shape)
print("labels 크기:", batch["labels"].shape)

## SFTConfig 설정

### LoRA 설정

- LoRAConfig 주요 매개변수

| 매개변수                | 의미/역할                                         | 주요 옵션·예시                                                                                                             |
| ------------------- | --------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- |
| **r**               | LoRA 어댑터의 랭크(정보량/두께). 값이 크면 표현력은 높아지지만 메모리를 더 사용하게 된다.<br>2B ~ 7B 모델의 경우 8이 가장 효율이 좋은 것으로 알려졌다.  | 8, 16, 32 등                                                                                                          |
| **lora\_alpha**     | - LoRA 어댑터의 출력 크기에 곱해주는 스케일링 계수. $\Delta W = BA$에 곱해주는 스케일링 계수. $\Delta W = \cfrac{\alpha}{r}BA$<br>- `BA`는 초기에 매우 작은 값으로 시작해서 초반 업데이트가 너무느리게 진행될 수있다. 이것을 보정해 학습 안정성을 높이는 값이다.| 16, 32, 64 등                                                                                                         |
| **lora\_dropout**   | 어댑터에만 적용되는 드롭아웃 확률. 과적합 방지                    | 0.05, 0.1 등                                                                                                          |
| **bias**            | 기존 모델의 bias 파라미터도 LoRA로 튜닝할지 여부               | "none"(권장), "all"                                                                                                    |
| **target\_modules** | LoRA를 어떤 레이어(부분)에 적용할지 지정                     | "q\_proj", "k\_proj", "v\_proj", "o\_proj"
| **task\_type**      | LoRA가 적용될 문제 유형(파인튜닝 목적)                      | "CAUSAL\_LM"(생성)<br>"SEQ\_CLS"(분류)<br>"SEQ\_2\_SEQ\_LM"(번역/요약)<br>"TOKEN\_CLS"(토큰분류)<br>"QUESTION\_ANSWERING"(질의응답) |

In [ ]:
###############################
# Peft(LoRA) 어뎁터 설정
###############################

from peft import LoraConfig
from trl import SFTConfig

peft_config = LoraConfig(
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

### SFTConfig 설정
- 전체 학습관련 설정

### SFTConfig 주요 매개변수

| 매개변수                                | 설명                                                  | 주요 예시 값/옵션                       |
| ----------------------------------- | --------------------------------------------------- | ---------------------------------------       |
| **output\_dir**                     | 결과 모델/로그를 저장할 경로 또는 저장소 ID                          | `"./results"`                |
| **num\_train\_epochs**              | 전체 데이터를 몇 번 반복 학습할지(에포크 수)                          | `3`, `5`                    |
| **per\_device\_train\_batch\_size** | 각 GPU(디바이스)에서 한 번에 입력할 데이터 수(배치 크기)                 | `2`, `4`, `8`            |
| **gradient\_accumulation\_steps**   | 여러 미니배치를 모아 한 번에 업데이트(실질 배치 크기 키우기)                 | `1`, `2`, `4`        |
| **gradient\_checkpointing**         | 메모리 절약 기능(필요할 때만 중간 계산값 저장)                         | `True`, `False`            |
| **optim**                           | 최적화 알고리즘(학습 방법)                                     | `"adamw_torch_fused"`              |
| **logging\_steps**                  | 몇 step마다 로그를 출력할지                                   | `10`, `50`, `100`                   |
| **save\_strategy**                  | 모델 저장 방식(주기)                                        | `"steps"`, `"epoch"`                  |
| **save\_steps**                     | 몇 step마다 모델을 저장할지                                   | `50`, `100`                         |
| **bf16**                            | bfloat16 연산 사용(GPU 메모리 절약)                          | `True`, `False`                      |
| **learning\_rate**                  | 파라미터 업데이트 속도(학습률)                                   | `1e-4`, `5e-5`                   |
| **max\_grad\_norm**                 | 그래디언트 클리핑 임계값(학습 안정화)                               | `0.3`, `1.0`                  |
| **warmup\_ratio**                   | 워밍업 단계 비율(초기 학습률 천천히 증가)                            | `0.03`, `0.1`                |
| **lr\_scheduler\_type**             | 학습률 조정 방식                                           | `"constant"`, `"linear"`               |
| **push\_to\_hub**                   | 학습 결과를 Hugging Face Hub로 업로드할지 여부                   | `True`, `False`                  |
| **hub\_model\_id**                  | 업로드할 Hugging Face Hub 저장소 ID                        |                                        |
| **hub\_token**                      | Hugging Face Hub 인증 토큰 사용 여부                        | `True`                                 |
| **remove\_unused\_columns**         | 학습에 안 쓰는 데이터 컬럼 자동 제거 여부                            | `True`, `False`               |
| **dataset\_kwargs**                 | 데이터셋 추가 옵션(딕셔너리 형태)                                 | `{"skip_prepare_dataset": True}` |
| **report\_to**                      | 학습 로그를 기록할 대상(예: wandb, tensorboard, 빈 리스트면 기록 안 함) | `None`, `["wandb"]`        |
| **max\_length**                     | 한 입력에 허용되는 최대 토큰(단어) 수 ()                             | `2048`, `4096`, `8192`          |
| **label\_names**                    | Trainer가 label로 인식할 컬럼명                             | `["labels"]`                           |


In [ ]:
epochs = 10

# Hugginface Model-hub에 업로드할 모델 ID
user_id = "" # ****본인 Huggingface ID 입력****
model_id = "kanana-nano-2.1b-instruct-finace_news-finetuning-100"
args = SFTConfig(
    output_dir=model_id,                    # 학습 결과(체크포인트/최종 모델)를 저장할 디렉터리 경로
    save_strategy="steps",                  # 모델 저장 전략("steps": 스텝 간격으로 저장)
    save_steps=50,                          # 얼마나 자주 체크포인트를 저장할지(step 단위)

    num_train_epochs=epochs,                # 전체 데이터셋을 몇 번 반복 학습할지(에포크 수)
    per_device_train_batch_size=2,          # GPU 하나당 학습 배치 크기(micro-batch size)
    gradient_accumulation_steps=4,          # gradient를 몇 스텝 누적해서 업데이트할지(효과적 batch 증가) :contentReference[oaicite:1]{index=1}
    gradient_checkpointing=True,            # 활성값 저장 대신 필요할 때 재계산하여 메모리 절감 :contentReference[oaicite:2]{index=2}
                                            # 중간 활성값을 역전파 때 저장하지 않고 필요 시 재계산하여 GPU 메모리를 크게 줄이는 기법입니다

    optim="adamw_torch_fused",              # 옵티마이저 종류(AdamW 변형, PyTorch fused 구현-Fused 구현은 연산이 통합되어 더 빠르고 효율적입니다.)
    logging_steps=20,                       # 로깅(손실 등 출력)을 몇 스텝마다 할지
    learning_rate=1e-4,                     # 학습률(learning rate)
    warmup_ratio=0.03,                      # warmup 비율(전체 step의 몇 %를 lr warmup으로 쓸지-전체 스텝의 3%를 warmup으로 쓰며, 이 구간 동안 lr이 낮은 값에서 점점 증가합니다.)
    max_grad_norm=0.3,                      # gradient clipping 최대 노름(norm) 값(그래디언트 폭주 방지)
    bf16=True,                              # bfloat16 mixed precision 활성화(메모리/연산 최적화)
    lr_scheduler_type="constant",           # learning rate scheduler(조정) 유형전략(여기서는 학습률 일정 유지-"constant"는 warmup 후 일정한 lr을 유지합니다.)

    push_to_hub=True,                       # 학습 완료 후 Hugging Face Hub에 업로드 여부
    hub_model_id=f"{user_id}/{model_id}",       # Hub에 업로드할 모델 리포지토리 ID
    hub_token=True,                         # Hub 업로드 시 사용할 토큰 정보

    remove_unused_columns=False,            # 기본적으로 Hugging Face Tokenizer/Trainer는 모델에 필요 없는 컬럼을 제거합니다. False면 제거하지 않고 그대로 두며, 사전에 collate_fn에서 필요한 처리가 된 경우에 쓰입니다.
    dataset_kwargs={"skip_prepare_dataset": True},  # SFTTrainer가 자동으로 데이터 전처리/packing을 수행하는 단계를 건너뛰게 합니다. 이미 데이터셋이 원하는 형식으로 준비된 경우 True로 설정합니다.
    max_length=max_seq_length,              # 최대 토큰 길이 제한(max_length)
    label_names=["labels"],                 # loss 계산 시 사용할 레이블 컬럼 이름
    report_to=None                          # 로깅/모니터링 대상(off이면 wandb/tensorboard 등 비활성)
)


## 학습하기

In [ ]:
from trl import SFTTrainer
# Trainer 객체 생성 
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=trainset,
    data_collator=collate_fn,
    peft_config=peft_config
)

In [ ]:
#############################
#  학습 시작
#############################
trainer.train()

# 테스트

## LoRA 파인튜닝 모델 사용법

- 위 과정에서 LoRA(Low-Rank Adaptation) 어댑터를 파인튜닝하여 Hugging Face Hub에 업로드한 상태
- LoRA는 전체 모델을 학습하는 대신 작은 어댑터만 학습하는 효율적인 방법

- **사용 시 필요한 구성 요소**
    - Base Model (기본 모델)
    - LoRA Adapter (파인튜닝된 어댑터)
    - 이 둘을 merge해서 사용한다. 
- **사용 방법 2가지**
    1. **개별 로드 후 병합**
        - 기본 모델과 LoRA 어댑터를 각각 다운로드
        - 런타임에서 두 구성 요소를 병합하여 사용

    2. **병합된 모델 직접 사용**
        - 미리 병합된 상태의 모델을 다운로드
        - 바로 사용 가능
    - 첫 번째 방법은 메모리 효율성과 유연성을 제공하고, 두 번째 방법은 사용 편의성을 제공합니다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel

user_id = "" # ****본인 Huggingface ID 입력****
base_model_id = "kakaocorp/kanana-nano-2.1b-instruct"
lora_model_id = f"{user_id}/kanana-nano-2.1b-instruct-finace_news-finetuning-100"

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
llama_input_creator = InputPromptCreator(tokenizer)

# base model 불러오기
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    dtype=torch.bfloat16
)

# base model에 LoRA adapter 추가
model = PeftModel.from_pretrained(base_model, lora_model_id)

# 파이프라인 생성
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

In [ ]:
news_title = "'반도체 초호황'…삼성전자 사상 첫 분기 영업익 20조 넘나"
news_document = """삼성전자가 지난해 4분기 사상 첫 분기 영업이익 20조를 달성할 수 있을지 업계와 시장의 이목이 집중되고 있다. 인공지능(AI) 수요 확대에 따른 메모리 초호황이 전례 없는 ‘역대급’ 실적을 만들고 있기 때문이다.
4일 업계에 따르면 삼성전자(005930)는 오는 8일 오전 지난해 4분기 잠정실적을 공개한다. 금융정보업체 에프엔가이드의 집계를 보면, 삼성전자 실적 컨센서스는 매출 89조2173억원, 영업이익 16조4545억원이다. 다만 일부 증권사들은 20조원 넘는 영업이익을 점치고 있다. 메모리 초호황기였던 2018년 3분기 당시 17조5700억원을 뛰어넘는 역대 최대 실적을 달성할 수 있다는 의미다. 지난해 3분기 1년3개월 만에 ‘10조 클럽’에 복귀한 이후 불과 한 개 분기 만에 ‘20조 클럽’에 입성할 가능성이 커졌다.
호실적을 견인하고 있는 것 D램, 낸드플래시 등 범용 메모리 가격의 급등이다. 시장조사업체 D램익스체인지에 따르면, 지난해 12월 기준 PC용 D램 범용 제품(DDR4 8Gb 1Gx8) 평균 고정거래 가격은 9.3달러로 집계됐다. 2024년 말(1.35달러)과 비교해 6.9배 급등했다. 트렌드포스는 서버용 DDR5와 5세대 고대역폭메모리(HBM3E) 간 가격 격차가 올해 말엔 4~5배에서 1~2배 수준까지 줄어들 것이라고 예상했다.
이는 삼성전자, SK하이닉스, 마이크론 등 메모리 3사가 AI 시대 들어 수익성이 높은 HBM의 생산 비중을 높이는 와중에 범용 메모리 수요가 갑자기 폭증했기 때문이다. 특히 삼성전자는 압도적인 D램 생산능력으로 이익 규모를 늘린 것으로 추정된다. 업계에 따르면 삼성전자의 범용 D램 생산능력(월 웨이퍼 투입량 기준)은 약 50만5000장 수준이다. SK하이닉스와 마이크론은 각각 약 39만5000장, 29만5000장이다.
HBM 호조도 긍정적인 영향을 미쳤다. 경쟁사에 비해 HBM 시장에서 뒤처졌던 삼성전자는 재설계를 통해 성능을 끌어올렸고, 지난해 말 엔비디아의 HBM3E 품질 테스트를 통과했다. ‘아픈 손가락’ 파운드리 사업의 적자 폭도 1조원 내로 줄어들었다는 관측이 나온다. 올해 파운드리, 시스템LSI 등 비메모리 사업은 흑자 전환을 위한 반등에 박차를 가한다는 방침이다.
삼성전자는 메모리 초호황을 등에 업고 올해 내내 호실적을 낼 것으로 보인다. 산업계 한 관계자는 “삼성전자가 올해 영업이익 100조원을 돌파할 수 있다는 전망도 충분히 설득력 있게 받아들여지고 있다”고 말했다. 연간 영업이익 100조원은 전례가 없는 기록이다.
LG전자와 LG에너지솔루션도 각각 9일과 8일 잠정실적을 발표한다. 에프엔가이드에 따르면, LG전자의 실적 컨센서스는 매출 23조5748억원, 영업손실 76억원이다. 전 사업부 차원의 희망퇴직 등 비용 효율화에 따른 것으로 읽힌다.
LG에너지솔루션도 전기차 캐즘 충격파에 부진한 성적표를 받아들 전망이다. 다시 적자로 전환할 게 유력하다. LG에너지솔루션은 최근 들어서만 약 13조6000억원 규모의 배터리 계약을 해지했다."""

message = llama_input_creator.create_pipeline_prompt(news_title=news_title, news_document=news_document)

In [ ]:
# news_title = "\"구글 제치고 압도적 1위\" 네이버 큰일 했네"
# news_document = """네이버 검색 점유율이 3년만에 60%를 넘겨 구글을 제치고 압도적 1위를 차지 토종 포털사이트의 자존심을 지킨 것으로 나타났다.
# 4일 시장조사업체 ‘인터넷트렌드’에 따르면 지난해 네이버 국내 검색 점유율은 평균 62.86%로 집계됐다.
# 전년도인 2024년 58.14%와 비교해 4.72% 증가한 수치로 네이버 점유율이 60%를 넘긴 것은 2022년 61.20%를 기록한 이후 3년 만이다.
# 2위인 구글은 전년 동기 대비 3.45% 감소한 29.55%의 검색 점유율을 보였다.
# 국내와 해외를 대표하는 양대 플랫폼의 검색 점유율 격차가 1년 사이 더 벌어진 것으로 국내 검색 시장에서 네이버 지배력이 더 확고해진 것으로 해석된다.
# 3위부터는 2위와 큰 격차를 보여 영향력이 미미했다. 마이크로소프트(MS) 검색 엔진 빙(Bing)은 전년 점유율 2.91% 대비 소폭 상승한 3.12%로 3위를, 다음은 전년 3.72% 대비 소폭 감소한 2.94%로 4위를 각각 차지했다.
# 특히 줌과 야후 등 기타 검색 사이트는 점유율 1%를 넘지 못했다.
# 업계에서는 네이버의 반등에 대해 검색 신뢰도 향상 차원에서 네이버가 다양한 기술을 시도한 노력의 결과라고 풀이했다.
# 특히 네이버가 작년 신규 출시한 AI 검색 ‘AI 브리핑’과 맞물려 검색 접촉 횟수가 더 늘었다는 분석이다."""

# message = llama_input_creator.create_pipeline_prompt(news_title=news_title, news_document=news_document)

In [ ]:
message

In [ ]:
response = pipe(message, max_new_tokens=1000)

In [ ]:
response

In [ ]:
print(response[0]['generated_text'][-1]["content"])

In [ ]:
#####################################
# Merge된 모델 한번에 받아오기
#####################################

from peft import AutoPeftModelForCausalLM
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(lora_model_id, device_map="auto", dtype=torch.bfloat16)
pipe2 = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

In [ ]:
response2 = pipe2(message, max_new_tokens=1000)

In [ ]:
print(response[0]['generated_text'][-1]["content"])

In [ ]:
class Predict:

    def __init__(self, pipeline):
        self.pipeline = pipeline
        self.prompt_creator = InputPromptCreator(pipeline.tokenizer)

    def __call__(self, title: str, content: str) -> str:
        """뉴스 제목과 뉴스 내용을 받아서 프롬프트 생성 후 pipeline에 전달하여 응답 내용만 추출 해서 반환

        Args:
            title (str): 뉴스제목
            content (str): 뉴스내용

        Returns:
            str: LLM 응답
        """
        message = self.prompt_creator.create_pipeline_prompt(title, content)
        response = self.pipeline(message)
        return response[0]['generated_text'][-1]['content']

In [ ]:
predict = Predict(pipe)
predict(news_title, news_document)